In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, lit, concat, when

spark_session = SparkSession.builder.enableHiveSupport().master("local").getOrCreate()

path = '/data/twitter/twitter_sample2.txt'

graph = spark_session.read.text(path)

source = 12
target = 34

In [ ]:
graph = graph\
    .withColumn('tmp', split('value', '\t'))\
    .withColumn('source', col('tmp').getItem(0))\
    .withColumn('target', col('tmp').getItem(1))\
    .drop('value', 'tmp')\
    .cache()

In [ ]:
paths = graph\
    .where(col('source') == source)\
    .withColumn('v', col('source'))\
    .withColumn('dist', lit(0))\
    .limit(1)\
    .drop('target', 'source')\
    .select(col('v'), col('dist'))\
    .cache()

In [ ]:
dist = 0

while True:
    paths = paths\
        .join(graph, [col('v') == col('source'), col('v') != col('target'), col('dist') == dist], 'inner')\
        .withColumn('v', col('target'))\
        .withColumn('dist', col('dist') + 1)\
        .drop('target', 'source')\
        .select(col('v'), col('dist'))\
        .distinct()\
        .union(paths)\
        .persist()
    
    res = paths.where(col('v') == target).select(col('dist'))
    
    dist += 1
    if res.count() > 0:
        break
        
print(dist)